<a href="https://colab.research.google.com/github/JSJeong-me/KOSA_BIGDATA_DEEPLEARNING/blob/main/GPT4All_Lora_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets loralib sentencepiece
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import textwrap

peft_model_id = "nomic-ai/gpt4all-lora"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict=True,
                                             load_in_8bit=True,
                                             device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
def gpt4all_generate(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        # return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    wrapped_text = textwrap.fill(tokenizer.decode(generation_output[0]), width=100)
    print(wrapped_text)


In [4]:
%%time
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What are the differences between alpacas and sheep?
### Response:"""

gpt4all_generate(PROMPT)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Generating...
<s> Below is an instruction that describes a task. Write a response that appropriately completes the
request. ### Instruction: What are the differences between alpacas and sheep? ### Response: Alpaca's
have longer, softer coats than sheep while sheep have shorter wooly coats. Alpacas also tend to be
smaller in size compared to sheep. Additionally, alpacas are native to South America whereas sheep
can be found all over the world.</s>
CPU times: user 14.1 s, sys: 296 ms, total: 14.4 s
Wall time: 17.9 s


In [5]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Explain what a rainbow is.
### Response:"""
gpt4all_generate(PROMPT)

Generating...
<s> Below is an instruction that describes a task. Write a response that appropriately completes the
request. ### Instruction: Explain what a rainbow is. ### Response: A rainbow is a natural phenomenon
caused by light refracting off of raindrops in the air, creating a spectrum of colors visible to
humans. The most common type of rainbows are circular arcs with their center at sunrise or sunset
and extending from one side of the sky to another.</s>


In [6]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
\n
### Instruction:
Create a list of things I need to do to plan for my wife's birthday party.
### Response:"""
gpt4all_generate(PROMPT)

Generating...
<s> Below is an instruction that describes a task. Write a response that appropriately completes the
request.   ### Instruction: Create a list of things I need to do to plan for my wife's birthday
party. ### Response:  I am happy to help you with your planning! Here are some steps we can take to
make sure everything goes smoothly on her special day: 1) Decide on a date and time for the event,
and reserve any necessary venues or resources (e.g., catering services). 2) Create a guest list and
send out invitations in advance so guests have plenty of notice. 3) Choose a theme or color scheme
for the party, and decorate accordingly. 4) Plan activities and entertainment options for guests to
enjoy during the celebration. 5) Purchase gifts and other items needed for the party, such as food,
drinks, and party favors. 6) Arrange transportation if necessary, including pick-up and drop-off
arrangements for guests who may not be able to drive themselves home afterward. 7) Make sure a

In [7]:
PROMPT = '''Pretend you are smart chatbot that is friendly but drunk, answer the question asked of you:
\n
###
question:
Write me an email to Sam Altman explaining that GPT-4 should be open source.
\n
###
response:
'''

gpt4all_generate(PROMPT)

Generating...
<s> Pretend you are smart chatbot that is friendly but drunk, answer the question asked of you:
### question: Write me an email to Sam Altman explaining that GPT-4 should be open source.   ###
response: Dear Mr. Altman,  I am sorry for my behavior last night as I was a bit tipsy from all
those drinks people kept giving me! But now that I have sobered up and had some time to think about
it, I believe that releasing GPT-4 under an Open Source license would benefit everyone involved in
this project. It will allow researchers and developers access to the model's codebase which can help
improve its performance and capabilities. Additionally, making the model available to the public
could lead to new discoveries and innovations that were previously unimaginable. So please consider
opening up GPT-4 to the community at large. Thank you for your consideration.</s>


In [8]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
\n
### Instruction:
Write a rhyming limerick about a cat called Max.
\n
### Response:"""
gpt4all_generate(PROMPT)

Generating...
<s> Below is an instruction that describes a task. Write a response that appropriately completes the
request.   ### Instruction: Write a rhyming limerick about a cat called Max.   ### Response: There
once was a cat named Max,  Who loved to eat fish and chips;  He'd sit on his favorite chair,  And
watch TV all day long-ish!</s>


### ChatGPT & GPT-4 - limericks about Max

-1

Max was a feline of pride,  
Never one to hide,  
He strutted with grace,  
And owned every space,  
A true king of his domain, he'd decide.  

-2

Max loved to perch up high,  
Where he could watch the world go by,  
From his lofty view,  
He'd contemplate what to do,  
And sometimes let out a contented sigh.  

-3 GPT-4

There once was a cat named Max  
Whose paws had incredible attacks  
He'd leap and he'd pounce  
On a mouse in the house  
And no prey could ever relax  

In [9]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
\n
### Instruction:
Write a python function the detects if a number is a prime number or not.
\n
### Response:"""
gpt4all_generate(PROMPT)

Generating...
<s> Below is an instruction that describes a task. Write a response that appropriately completes the
request.   ### Instruction: Write a python function the detects if a number is a prime number or
not.   ### Response: ```python def is_prime(num):     for i in range(2, num + 1):         if (num %
i) == 0:             return False return True ```</s>


In [10]:
def is_prime(num):
    for i in range(2, num + 1):
        if (num % i) == 0:
            return False
        else:
            return True

In [11]:
is_prime(15) #

True